In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cta-ridership-data/CTA_-_Ridership_-_Daily_Boarding_Totals_20241108.csv
/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv
/kaggle/input/cta-st-info/CTA_-_System_Information_-_List_of__L__Stops_20241108.csv


In [3]:
cta_by_st = pd.read_csv('/kaggle/input/cta-ridership-data/CTA_-_Ridership_-__L__Station_Entries_-_Daily_Totals_20241108.csv')

print(cta_by_st.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides'], dtype='object')


W=Weekday, A=Saturday, U=Sunday/Holiday

In [4]:
# Ensure 'date' is in datetime format
cta_by_st['date'] = pd.to_datetime(cta_by_st['date'])

# Create a 'year' column by extracting the year from the 'date' column
cta_by_st['year'] = cta_by_st['date'].dt.year

# Group by station and year, and sum the ridership ('rides')
cta_yearly = cta_by_st.groupby(['stationname', 'year'])['rides'].sum().reset_index()

# Display the first few rows of the new dataframe
print(cta_yearly.head())

  stationname  year   rides
0        18th  2001  256183
1        18th  2002  251387
2        18th  2003  251773
3        18th  2004  236536
4        18th  2005  320783


In [5]:
cta_st_info = pd.read_csv('/kaggle/input/cta-st-info/CTA_-_System_Information_-_List_of__L__Stops_20241108.csv')

print(cta_st_info.columns)

Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [6]:
print(cta_by_st.columns)
print(cta_st_info.columns)

Index(['station_id', 'stationname', 'date', 'daytype', 'rides', 'year'], dtype='object')
Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [7]:
# Rename 'stationname' in cta_by_st to 'STATION_NAME' to match cta_st_info
cta_by_st.rename(columns={'stationname': 'STATION_NAME'}, inplace=True)

# Merge the two DataFrames on 'STATION_NAME'
cta_stn_merged = pd.merge(cta_by_st, cta_st_info[['STATION_NAME', 'RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']], 
                          on='STATION_NAME', how='left')

# Check the first few rows of the merged DataFrame
print(cta_stn_merged.head())

   station_id      STATION_NAME       date daytype  rides  year    RED   BLUE  \
0       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
1       41280    Jefferson Park 2017-12-22       W   6104  2017  False   True   
2       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
3       41000  Cermak-Chinatown 2017-12-18       W   3636  2017   True  False   
4       40280      Central-Lake 2017-12-02       A   1270  2017    NaN    NaN   

       G    BRN      P   Pexp      Y    Pnk      O  
0  False  False  False  False  False  False  False  
1  False  False  False  False  False  False  False  
2  False  False  False  False  False  False  False  
3  False  False  False  False  False  False  False  
4    NaN    NaN    NaN    NaN    NaN    NaN    NaN  


In [8]:
# Export the merged DataFrame to a CSV file
cta_stn_merged.to_csv('cta_stn_merged.csv', index=False)

In [9]:
# Define the line columns to aggregate
line_columns = ['RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']

# Group by year and station and sum the 'rides' and the line columns
cta_ridership_total = cta_stn_merged.groupby(['year', 'STATION_NAME'], as_index=False)[['rides'] + line_columns].sum()

# If 'RED' and other line columns exist, apply the transformation
for line in line_columns:
    cta_ridership_total[line] = cta_ridership_total[line].apply(lambda x: 1 if x > 0 else 0)

# Check the result
cta_ridership_total


,year,STATION_NAME,rides,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O
0,2001,18th,512366,0,0,0,0,0,0,0,1,0
1,2001,35-Bronzeville-IIT,567806,0,0,0,0,0,0,0,0,0
2,2001,35th/Archer,1254906,0,0,0,0,0,0,0,0,1
3,2001,43rd,495980,0,0,1,0,0,0,0,0,0
4,2001,47th-Dan Ryan,833311,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3424,2024,Western-Cermak,189322,0,0,0,0,0,0,0,0,0
3425,2024,Western-Forest Park,134776,0,0,0,0,0,0,0,0,0
3426,2024,Western-Orange,482034,0,0,0,0,0,0,0,0,0
3427,2024,Western/Milwaukee,653798,0,0,0,0,0,0,0,0,0


In [10]:
# Create 'active_lines' column by summing across the line columns
cta_ridership_total['active_lines'] = cta_ridership_total[line_columns].sum(axis=1)

# Now calculate the correlation between 'rides' and 'active_lines'
correlation = cta_ridership_total['rides'].corr(cta_ridership_total['active_lines'])
print(f"Correlation between ridership and number of active lines: {correlation}")


Correlation between ridership and number of active lines: 0.5622907298577524


The correlation of 0.562 between ridership and the number of active lines indicates a moderate positive relationship. This means that as the number of active lines at a station increases, the ridership tends to increase as well, but the correlation is not very strong. There may be other factors influencing ridership that are not captured by the number of active lines alone.

In [11]:
# Data for train line capacities
# Define the train lines and their respective capacities
train_lines = {
    'RED': {'typical_capacity': 8, 'max_length': 8},
    'BLUE': {'typical_capacity': 8, 'max_length': 8},
    'G': {'typical_capacity': 8, 'max_length': 8},
    'BRN': {'typical_capacity': 6, 'max_length': 6},
    'P': {'typical_capacity': 8, 'max_length': 8},
    'Pexp': {'typical_capacity': 8, 'max_length': 8},
    'Y': {'typical_capacity': 2, 'max_length': 2},
    'Pnk': {'typical_capacity': 8, 'max_length': 8},
    'O': {'typical_capacity': 8, 'max_length': 8}
}

# Convert this dictionary into a DataFrame for easier handling
line_capacity_df = pd.DataFrame.from_dict(train_lines, orient='index')
line_capacity_df.reset_index(inplace=True)
line_capacity_df.rename(columns={'index': 'Line'}, inplace=True)


line_capacity_df

,Line,typical_capacity,max_length
0,RED,8,8
1,BLUE,8,8
2,G,8,8
3,BRN,6,6
4,P,8,8
5,Pexp,8,8
6,Y,2,2
7,Pnk,8,8
8,O,8,8


In [12]:
print(cta_st_info.columns)

Index(['STOP_ID', 'DIRECTION_ID', 'STOP_NAME', 'STATION_NAME',
       'STATION_DESCRIPTIVE_NAME', 'MAP_ID', 'ADA', 'RED', 'BLUE', 'G', 'BRN',
       'P', 'Pexp', 'Y', 'Pnk', 'O', 'Location'],
      dtype='object')


In [13]:
# Define the columns related to the train lines
train_lines_columns = ['RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']

# Convert the relevant columns to integer type to ensure proper summing
cta_st_info[train_lines_columns] = cta_st_info[train_lines_columns].astype(int)

# Create the 'total_num_lines' column by summing the values across the selected columns
cta_st_info['total_num_lines'] = cta_st_info[train_lines_columns].sum(axis=1)

# Display the updated DataFrame with the new column
cta_st_info

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,total_num_lines
0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,0,0,0,0,0,0,0,1,0,"(41.85182, -87.745336)",1
1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,0,0,0,0,0,0,0,1,0,"(41.853839, -87.714842)",1
2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,0,0,1,0,0,0,0,0,0,"(41.778943, -87.644244)",1
3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,0,1,0,0,0,0,0,0,0,"(41.984246, -87.838028)",1
4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,0,1,0,0,0,0,0,0,0,"(41.87592, -87.659458)",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,30125,N,North/Clybourn (Howard-bound),North/Clybourn,North/Clybourn (Red Line),40650,False,1,0,0,0,0,0,0,0,0,"(41.910655, -87.649177)",1
298,30172,S,O'Hare (Forest Pk-bound),O'Hare,O'Hare (Blue Line),40890,True,0,1,0,0,0,0,0,0,0,"(41.97766526, -87.90422307)",1
299,30045,S,Cumberland (Forest Pk-bound),Cumberland,Cumberland (Blue Line),40230,True,0,1,0,0,0,0,0,0,0,"(41.984246, -87.838028)",1
300,30388,W,Damen (Harlem-bound),Damen,Damen (Green Line),41710,True,0,0,1,0,0,0,0,0,0,"(41.88497405, -87.67689149)",1


In [16]:
# Sum the specified columns for each row and create a new column 'total_lines'
cta_st_info['total_lines'] = cta_st_info[['RED', 'BLUE', 'G', 'BRN', 'P', 'Pexp', 'Y', 'Pnk', 'O']].sum(axis=1)

# Display the updated DataFrame with the new column
cta_st_info[['STATION_NAME', 'total_lines']]

,STOP_ID,DIRECTION_ID,STOP_NAME,STATION_NAME,STATION_DESCRIPTIVE_NAME,MAP_ID,ADA,RED,BLUE,G,BRN,P,Pexp,Y,Pnk,O,Location,total_num_lines,total_lines
0,30082,E,Cicero (Loop-bound),Cicero,Cicero (Pink Line),40420,True,0,0,0,0,0,0,0,1,0,"(41.85182, -87.745336)",1,1
1,30151,E,Central Park (Loop-bound),Central Park,Central Park (Pink Line),40780,True,0,0,0,0,0,0,0,1,0,"(41.853839, -87.714842)",1,1
2,30184,W,Halsted/63rd (Ashland-bound),Halsted,Halsted (Green Line),40940,True,0,0,1,0,0,0,0,0,0,"(41.778943, -87.644244)",1,1
3,30044,N,Cumberland (O'Hare-bound),Cumberland,Cumberland (Blue Line),40230,True,0,1,0,0,0,0,0,0,0,"(41.984246, -87.838028)",1,1
4,30092,E,Racine (O'Hare-bound),Racine,Racine (Blue Line),40470,False,0,1,0,0,0,0,0,0,0,"(41.87592, -87.659458)",1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,30125,N,North/Clybourn (Howard-bound),North/Clybourn,North/Clybourn (Red Line),40650,False,1,0,0,0,0,0,0,0,0,"(41.910655, -87.649177)",1,1
298,30172,S,O'Hare (Forest Pk-bound),O'Hare,O'Hare (Blue Line),40890,True,0,1,0,0,0,0,0,0,0,"(41.97766526, -87.90422307)",1,1
299,30045,S,Cumberland (Forest Pk-bound),Cumberland,Cumberland (Blue Line),40230,True,0,1,0,0,0,0,0,0,0,"(41.984246, -87.838028)",1,1
300,30388,W,Damen (Harlem-bound),Damen,Damen (Green Line),41710,True,0,0,1,0,0,0,0,0,0,"(41.88497405, -87.67689149)",1,1


Unfortunately, it seems the data is not correct for the Chicago train stations. 

In [21]:
import re
# Define the train lines and their abbreviations
train_lines = {
    'RED': 'Red',
    'BLUE': 'Blue',
    'G': 'Green',
    'BRN': 'Brown',
    'P': 'Purple',
    'Pexp': 'Purple Express',
    'Y': 'Yellow',
    'Pnk': 'Pink',
    'O': 'Orange'
}

# Function to map stations to train lines
def map_train_lines(station_desc):
    # Convert the station description to lowercase for easier matching
    station_desc = station_desc.lower()
    # Initialize a list to hold the train lines for this station
    mapped_lines = []
    
    # Loop through the dictionary of train lines and check if they are mentioned in the station description
    for col, line_name in train_lines.items():
        # Check if the line is mentioned in the station description
        if re.search(r'\b' + re.escape(line_name.lower()) + r'\b', station_desc):
            mapped_lines.append(col)  # Append the column name if the line is found
    
    return mapped_lines

# Apply the function to the 'STATION_DESCRIPTIVE_NAME' column and create the new train line columns
for idx, row in cta_st_info.iterrows():
    # Get the list of train lines for this station
    train_lines_for_station = map_train_lines(row['STATION_DESCRIPTIVE_NAME'])
    
    # Set the corresponding columns to 1 based on the lines found
    for line in train_lines_for_station:
        cta_st_info.at[idx, line] = 1

# Fill non-1 values with 0 in the specified columns
for col in train_lines.keys():
    cta_st_info[col] = cta_st_info[col].apply(lambda x: 1 if x == 1 else 0)




In [27]:
# Get summary statistics for 'num_lines'
range_summary = cta_st_info['num_lines'].describe()

# Display the minimum and maximum values
min_num_lines = cta_st_info['num_lines'].min()
max_num_lines = cta_st_info['num_lines'].max()

print(f"Summary statistics for the number of lines per station:\n{range_summary}")
print(f"\nMinimum number of lines per station: {min_num_lines}")
print(f"Maximum number of lines per station: {max_num_lines}")

Summary statistics for the number of lines per station:
count    302.000000
mean       1.403974
std        1.006153
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        6.000000
Name: num_lines, dtype: float64

Minimum number of lines per station: 1
Maximum number of lines per station: 6


**Summary statistics for the number of lines per station:**
* count    302.000000
* mean       1.403974
* std        1.006153
* min        1.000000
* 25%        1.000000
* 50%        1.000000
* 75%        1.000000
* max        6.000000 

Name: num_lines, dtype: float64


* Minimum number of lines per station: 1
* Maximum number of lines per station: 6


In [28]:
# Create boolean columns for each direction
cta_st_info['direction_N'] = cta_st_info['DIRECTION_ID'].apply(lambda x: 1 if x == 'N' else 0)
cta_st_info['direction_S'] = cta_st_info['DIRECTION_ID'].apply(lambda x: 1 if x == 'S' else 0)
cta_st_info['direction_E'] = cta_st_info['DIRECTION_ID'].apply(lambda x: 1 if x == 'E' else 0)
cta_st_info['direction_W'] = cta_st_info['DIRECTION_ID'].apply(lambda x: 1 if x == 'W' else 0)

# Drop the original 'DIRECTION_ID' column if no longer needed
cta_st_info = cta_st_info.drop(columns=['DIRECTION_ID'])

In [30]:
# Check for station pairs with different 'num_lines' values
num_lines_discrepancies = cta_st_info.groupby(['STATION_NAME', 'STATION_DESCRIPTIVE_NAME'])['num_lines'].nunique().reset_index()
# Filter for stations where there is more than one unique 'num_lines' value
num_lines_discrepancies = num_lines_discrepancies[num_lines_discrepancies['num_lines'] > 1]

# Display the stations with discrepancies
if not num_lines_discrepancies.empty:
    print("Stations with different 'num_lines' values:")
    print(cta_st_info[cta_st_info['STATION_NAME'].isin(num_lines_discrepancies['STATION_NAME'])])
else:
    print("No discrepancies found in 'num_lines' values for stations.")

No discrepancies found in 'num_lines' values for stations.


In [33]:
# Renaming columns with mapping
cta_st_info = cta_st_info.rename(columns={
    'STOP_ID': 'stop_id',
    'STOP_NAME': 'stop_name',
    'STATION_NAME': 'station_name',
    'STATION_DESCRIPTIVE_NAME': 'station_descriptive_name',
    'MAP_ID': 'map_id',
    'ADA': 'ada',
    'RED': 'red',
    'BLUE': 'blue',
    'G': 'g',
    'BRN': 'brn',
    'P': 'p',
    'Y': 'y',
    'Pnk': 'pnk',
    'O': 'o',
    'Location': 'location',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'num_lines': 'num_lines',
    'direction_N': 'direction_n',
    'direction_S': 'direction_s',
    'direction_E': 'direction_e',
    'direction_W': 'direction_w'
})


In [37]:
# Group data by station to merge rows and summarize the directions per station
cta_st_info = cta_st_info.groupby([
    'station_name', 'station_descriptive_name', 'map_id', 'ada', 'red', 'blue', 'g', 
    'brn', 'p', 'y', 'pnk', 'o', 'latitude', 'longitude'
]).agg({
    'direction_n': 'max',
    'direction_s': 'max',
    'direction_e': 'max',
    'direction_w': 'max',
    'num_lines': 'max'  
}).reset_index()

In [45]:
def split_p_column(df, express_stops):
    # Create new columns initialized to 0
    df['p'] = 0
    df['pexp'] = 0
    
    # Regular service: Set 'p' to 1 for stops in the regular route
    regular_stops = ['Linden', 'Central', 'Noyes', 'Foster', 'Davis', 'Dempster', 'Main', 'South Blvd', 'Howard']
    df['p'] = df['station_name'].apply(lambda x: 1 if x in regular_stops else 0)
    
    # Express service: Set 'pexp' to 1 for express stops during rush hours
    df['pexp'] = df['station_name'].apply(lambda x: 1 if x in express_stops else 0)
    
    return df

express_stops = ['Belmont', 'Wellington', 'Diversey', 'Fullerton', 'Armitage', 'Sedgwick', 'Chicago', 'Merchandise Mart',
                 'Clark/Lake', 'State/Lake', 'Washington/Wabash', 'Adams/Wabash', 'Harold Washington Library-State/Van Buren',
                 'LaSalle/Van Buren', 'Quincy', 'Washington/Wells']

stations_by_lines = split_p_column(stations_by_lines, express_stops)

In [48]:
# Reorder the columns so 'pexp' is next to 'p'
columns_order = ['station_name', 'station_descriptive_name', 'map_id', 'ada', 'red', 'blue', 'g', 'brn', 'p', 'pexp', 
                 'y', 'pnk', 'o', 'latitude', 'longitude', 'direction_n', 'direction_s', 'direction_e', 'direction_w', 'num_lines']

# Apply the column reordering to your dataframe
stations_by_lines = stations_by_lines[columns_order]

In [49]:
# Export the dataframe to a CSV file
stations_by_lines.to_csv('stations_by_lines.csv', index=False)